In [1]:
import pyltp
from gensim.models import Word2Vec
from pyltp import Segmentor
import jieba
from gensim.models.word2vec import LineSentence
from pyltp import  SentenceSplitter,NamedEntityRecognizer,Postagger,Parser,Segmentor
from gensim import models
import numpy as np

ModuleNotFoundError: No module named 'pyltp'

In [2]:
with open("news_corpus","r") as f:
    news = ''
    for line in f.readlines():
        news += line

In [3]:
cws_model = "ltp_data_v3.4.0/cws.model"
pos_model = "ltp_data_v3.4.0/pos.model"
par_model = "ltp_data_v3.4.0/parser.model"
ner_model = "ltp_data_v3.4.0/ner.model"

In [4]:
def get_word_list(sentence,model):
    #得到分词
    segmentor = Segmentor()
    segmentor.load(model)
    word_list = list(segmentor.segment(sentence))
    segmentor.release()
    return word_list

def get_postag_list(word_list,model):
    #得到词性标注
    postag = Postagger()
    postag.load(model)
    postag_list = list(postag.postag(word_list))
    postag.release()
    return postag_list

def get_parser_list(word_list,postag_list,model):
    #得到依存关系
    parser = Parser()
    parser.load(model)
    arcs = parser.parse(word_list,postag_list)
    arc_list = [(arc.head,arc.relation) for arc in arcs]
    parser.release()
    return arc_list

In [5]:
word_list = get_word_list(news,cws_model)

In [6]:
postag_list = get_postag_list(word_list,pos_model)

In [7]:
parser_list = get_parser_list(word_list,postag_list,par_model)

In [8]:
for i in range(len(word_list)):
    print(word_list[i],parser_list[i])

据 (18, 'ADV')
彭博社 (4, 'SBV')
15日 (4, 'ADV')
报道 (1, 'POB')
， (1, 'WP')
特朗普 (18, 'SBV')
正在 (18, 'ADV')
开始 (18, 'ADV')
为 (18, 'ADV')
他 (14, 'ATT')
的 (10, 'RAD')
2020年 (14, 'ATT')
竞选 (14, 'ATT')
活动 (9, 'POB')
做 (9, 'POB')
准备 (15, 'VOB')
。特朗普 (16, 'VOB')
表示 (0, 'HED')
: (18, 'WP')
“ (32, 'WP')
如果 (32, 'ADV')
2020年 (32, 'ADV')
除了 (32, 'ADV')
我 (25, 'ATT')
之外 (27, 'ATT')
的 (25, 'RAD')
任何人 (28, 'SBV')
接管 (23, 'POB')
， (23, 'WP')
将 (32, 'ADV')
会 (32, 'ADV')
出现 (18, 'VOB')
前所未有 (35, 'ATT')
的 (33, 'RAD')
股市 (36, 'SBV')
崩盘 (32, 'VOB')
! (32, 'WP')
” (32, 'WP')
特朗普 (40, 'SBV')
说 (32, 'COO')
： (40, 'WP')
“ (43, 'WP')
好 (40, 'VOB')
吧 (43, 'RAD')
， (43, 'WP')
我 (47, 'SBV')
想 (43, 'COO')
我 (51, 'ATT')
伟大 (51, 'ATT')
的 (49, 'RAD')
飞行员 (53, 'SBV')
会 (53, 'ADV')
知道 (47, 'VOB')
的 (53, 'RAD')
。 (18, 'WP')
他们 (57, 'SBV')
看到 (59, 'ATT')
的 (57, 'RAD')
情况 (63, 'SBV')
与 (63, 'ADV')
过去 (60, 'POB')
略 (63, 'ADV')
有 (18, 'COO')
不同 (63, 'VOB')
。 (18, 'WP')
我们 (69, 'SBV')
会 (69, 'ADV')
去 (69, 'ADV')
看 (18, 'COO')
的 (6

In [9]:
#加载预训练好的词向量模型
word2vec = Word2Vec.load("word2vec.model")

In [10]:
word2vec.most_similar("说",topn=10)

/Users/limingxiao/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.
/Users/limingxiao/anaconda3/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('表示', 0.8936804533004761),
 ('认为', 0.862716794013977),
 ('坦言', 0.8580939769744873),
 ('指出', 0.8195451498031616),
 ('看来', 0.7946385145187378),
 ('所说', 0.7787483930587769),
 ('透露', 0.7716749310493469),
 ('明说', 0.7267775535583496),
 ('称', 0.7238091826438904),
 ('介绍', 0.7030166387557983)]

# Get similar words

In [11]:
def get_similar_word_list(word,top=10): 
    #取得与word最相近的10个词
    word_list = []
    for w in word2vec.most_similar(word,topn=top):
        word_list.append(w[0])
    return word_list

In [12]:
def gather_similar_word(word): 
    #广度优先搜索 取得深度为10 与word 相近的所有词 
    words = [word]
    seen = set()
    depth = 10
    while words and depth > 0:
        word = words.pop(0)
        for w in word:
            if w not in seen:
                seen.add(w)
                words.append(get_similar_word_list(w,10))
            else:
                continue
        depth -= 1
    return list(seen)

In [13]:
spoken_words = gather_similar_word("说")

/Users/limingxiao/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  after removing the cwd from sys.path.
/Users/limingxiao/anaconda3/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [14]:
spoken_words.append("报道")  #人为加入未在预料中找到的表示说的词 因为这里我示范用的新闻和词向量训练的预料不同 所以手动加一下这个词

In [15]:
print(spoken_words)

['眼中', '称', '说', '指出', '坦言', '明说', '写道', '看来', '地说', '所说', '透露', '普遍认为', '告诉', '眼里', '直言', '强调', '文说', '说道', '武说', '表示', '提到', '正说', '介绍', '相信', '认为', '问', '报道']


# Get SBV relation in corpus

In [16]:
def find_spoken_word_id_and_sub(spoken_words,word_list,parser_list):
    #取得 新闻中 包含SBV关系 并且V表示的是说的意思 的主语和谓语的位置
    id_list = []
    for sub_id,parse_relation in enumerate(parser_list):
        index,relation = parse_relation
        if  relation == "SBV":
            spoken_word = word_list[index-1]
            if spoken_word in spoken_words:
                word_id = index-1
                id_list.append((sub_id,word_id))
    return id_list

In [17]:
spoken_words.append("报道")

In [18]:
idx=find_spoken_word_id_and_sub(spoken_words,word_list,parser_list)

In [19]:
idx

[(1, 3), (5, 17), (38, 39), (74, 76), (120, 126)]

In [20]:
def get_sentence(news,word_list,idx):
    #取得 说的内容 及SBV的宾语成分
    # idx 为表示说的词在新闻中的位置信息
    index = len("".join(word_list[:idx+1]))
    #寻找说这个词后 第一个句子 
    stop1 = news[index+1:].find("。")
    stop2 = news[index+1:].find("!")
    stop3 = news[index+1:].find("?")
    #检查是不是后面没有句子了 
    stop_list = [stop for stop in [stop1,stop2,stop3] if stop != -1]
    if stop_list is None:
        False
    #返回后面的第一个句子
    stop = min(stop_list)
    
    #检查表示说后面是否有双引号 引起来的句子
    begin = 9999
    end = 9999 
    if "“" in news[index+1:] and "”" in news[index+1:]:
        begin = news[index+1:].find("“")
        end = news[index+1:].find("”")
    
    #第一种情况 双引号在stop前面，表示说词后面跟的是双引号的句子 则双引号里的内容即为说的内容
    if begin < stop:
        result = news[index+1+begin:index+1+end+1]
    #第二种情况 双引号的内容在第一个句子后或无双引号，则说词后的句子即为说的内容。
    else:
        sentence = news[index+2:stop+index+2].strip()
        result =sentence
        sim = 1
        next_id = stop+index+2
        # 检查第二个句子是否也是说的内容，通过检查句子相似性来判断 若是相似度大于某个数值则表示 相近 这个句子也是说的内容
        # 如果相似度大于0.7表示 该句话和前一句内容相似 所以这句话 也为说的内容 继续检查下一句话
        while sim > 0.85 and next_id < len(news)-1:
            next_sentence_id = next_id
            if next_sentence_id < len(news)-1:
                next_sentence,next_id = get_next_sentence(news,next_sentence_id)
                sim = sentence_distance(sentence,next_sentence)
            if sim > 0.85:
                result += next_sentence
                sentence = next_sentence
            
    return result

In [21]:
def get_next_sentence(news,index):
    #取得 index 后面的第一句话
    stop1 = news[index+1:].find("。")
    stop2 = news[index+1:].find("!")
    stop3 = news[index+1:].find("?")
    stop_list = [stop for stop in [stop1,stop2,stop3] if stop != -1]
    if stop_list is None:
        False
    stop = min(stop_list)
    return news[index:index+stop+2],index+stop+2

In [22]:
def cut(sentence):
    return " ".join(jieba.cut(sentence))

In [23]:
def sentence_distance(sentence1,sentence2):
    #计算句子间的距离 这里使用余弦距离 句子的embedding使用的是句子中词向量相加
    word_list1 = cut(sentence1).split()
    word_list2 = cut(sentence2).split()
    
    vec_1 = 0
    vec_2 = 0
    # get representation of sentence 1
    for i in range(len(word_list1)):
        if word_list1[i] in word2vec.wv.vocab:
            vec_1 += word2vec.wv[word_list1[i]]
        
    # get representation of sentence 2
    for j in range(len(word_list2)):
        if word_list2[j] in word2vec.wv.vocab:
            vec_2 += word2vec.wv[word_list2[j]]
    
    return np.dot(vec_1,vec_2)/(np.linalg.norm(vec_1)*np.linalg.norm(vec_2))

In [24]:
def get_sub_and_view(idxs,news,word_list):
    # 取得主语 以及说的内容
    sub = []
    speech = []
    for sub_id,spoken_id in idxs:
        sub.append(word_list[sub_id])
        speech.append(get_sentence(news,word_list,spoken_id))
    return sub,speech

In [25]:
sub,speech = get_sub_and_view(idx,news,word_list)

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/c8/gq6b1ml94lx82wylsp11xt7c0000gn/T/jieba.cache
Loading model cost 0.940 seconds.
Prefix dict has been built succesfully.


In [26]:
for i in range(len(sub)):
    print(sub[i],speech[i])

彭博社 朗普正在开始为他的2020年竞选活动做准备。
特朗普 “如果2020年除了我之外的任何人接管，将会出现前所未有的股市崩盘!”
特朗普 “好吧，我想我伟大的飞行员会知道的。他们看到的情况与过去略有不同。我们会去看的。”
杜特尔特 美国希望战争一直持续，这样其他国家就会购买美国的飞机、军舰和子弹。一旦战争结束，美国将有一大群人因此失业，并且无法在国外销售任何产品。
杜特尔特 美国无法承受战争的结束，因为那样的话，很多美国人将失去工作,美国希望出售飞机、轮船和子弹。这就是美国希望一直能有战争，来保证美国人不会失去工作。
